# Makaan.com Scraping

By : Pratik Garai<br />
Email : pratikgarai0208@gmail.com

## Imports

In [1]:
from bs4 import BeautifulSoup
import requests
import time
from tqdm import tqdm
import time
import random
import re
import sqlite3
from sqlite3 import Error

In [2]:
URL = "https://www.makaan.com/price-trends"
CITIES = ["Chennai", "Mumbai", "Pune", "Puri", "Bangalore"]
TYPES = ["apartment", "villa", "plot", "builderfloor"] # Data type trend

## Core Methods

In [3]:
def scrape_page(url):
  '''
    Input a url and convert it to a BeautifulSoup object
  '''
  try:
    page = requests.get(url)
    html = BeautifulSoup(page.content, "html.parser")
    return (page.status_code, html)
  except Exception as e:
    raise Exception(f"Error in scraping page {url} : {e}")

def scrape_main_page(url) : 
  '''
    Scrape the entrypoint page table and create a list of dictionaries from it.
    Each dictionary contains : 
      * city_name : Name of the city
      * city_url : URL leading to localities in city
      * price_range_per_sq_ft : Price Range per sq. ft
      * avg_price_per_sq_ft : Average Price per sq. ft
      * price_rise : The price rise
      * type : The data trend type
    
    Inputs : 
      * url : The url of the main page
  '''
  status, page = scrape_page(URL)
  assert status == 200, f"Status {status}"
  tables = page.findAll("table")
  data = []

  # go through all tables on page
  # skipping the first table because it belongs to "Projects" section
  for table in tables[1:] : 
    table_rows = table.select("tbody tr")

    # go through in each row in table body
    for row in table_rows : 
      col = row.findAll("td")
      data.append({
          "city_name" : col[0].text,
          "city_url" : col[0].find("a").attrs["href"],
          "price_range_per_sq_ft" : col[1].text,
          "avg_price_per_sq_ft" : col[2].text,
          "price_rise" : col[3].text,
          "type" : table.attrs["data-trend-type"]
      })

  # return data
  return data

def scrape_cities(cities, city_urls) :
  '''
    Function to accept a list of cities and return their data in the form
    of a list of dictionaries

    The dataset has the following fields : 
      * city_name : Name of the city
      * city_url : URL leading to localities in city
      * locality_name : Name of the locality
      * price_range_per_sq_ft : Price Range per sq. ft
      * avg_price_per_sq_ft : Average Price per sq. ft
      * price_rise : The price rise
      * type : The data trend type
      * page : The page where the locality was present
    
    Inputs  : 
      * cities : A list/iterable of city names
      * city_urls : A map of city to city urls
  '''
  data = []

  for city in cities : 

    # extract city url
    assert city in city_urls, f"{city}'s url not found" 
    url = city_urls[city]
    status, page = scrape_page(url)
    assert status == 200, f"Status {status}"

    # check number of pages
    pagination = page.select(".pagination li")
    n_pages = 1
    if len(pagination) > 0 : 
      n_pages = int(pagination[-2].text)
    print(f"{n_pages} found for city : {city}")

    bar = tqdm(range(n_pages))
    for i in bar: 
      # scrape single locality page
      status, page = scrape_page(f"{url}?page={i+1}")
      assert status == 200, f"Status {status}"
      tables = page.findAll("table")
      bar.set_description(f"{i+1}/{n_pages} for city {city}")

      # go through all 4 data trend tables
      # skipping the last table because it belongs to "Projects" section
      for table in tables[:-1] : 
        table_rows = table.select("tbody tr")
        for row in table_rows : 
          col = row.findAll("td")
          data.append({
              "city_name" : city,
              "city_url" : url,
              "locality_name" : col[0].text,
              "price_range_per_sq_ft" : col[1].text,
              "avg_price_per_sq_ft" : col[2].text,
              "price_rise" : col[3].text,
              "type" : table.attrs["data-trend-type"],
              "page" : i+1
          })
      time.sleep(random.random()) # sleep random amount to avoid overloading server

  # return data
  return data

def get_row_list(row) :
  '''
    Takes in a dictionary and converts it to a row of csv
  '''
  r = []
  for i in row.copy().values() :
    if i == None : # handle NA values
      r.append("")
    else : 
      i = str(i)
      if "," in i : # handle commas
        r.append(f'"{str(i).strip()}"')
      else : 
        r.append(str(i).strip())
  return r

def save_to_csv(data, fname) : 
  '''
    Convert a list of dicts into a csv
  '''
  with open(fname, "w") as f : 
    f.write(",".join(list(data[0].keys())))
    f.write("\n")
    for row in data : 
      d = get_row_list(row)
      f.write(",".join(d))
      f.write("\n")

## Scrape Main Page

In [4]:
city_prices = scrape_main_page(URL)

In [5]:
city_prices[0]

{'city_name': 'Ahmedabad',
 'city_url': 'https://www.makaan.com/price-trends/property-rates-for-buy-in-ahmedabad',
 'price_range_per_sq_ft': ' 611 - 1,71,969 / sqft',
 'avg_price_per_sq_ft': ' 37,742.18 / sqft',
 'price_rise': '84.3%',
 'type': 'apartment'}

## Scrape Localities

In [6]:
CITY_URLS = {}
for city_price in city_prices : 
  city = city_price["city_name"]
  if city in CITIES : 
    if city not in CITY_URLS:
      CITY_URLS[city] = city_price["city_url"]

for city in CITIES : 
  print(city, CITY_URLS[city])

Chennai https://www.makaan.com/price-trends/property-rates-for-buy-in-chennai
Mumbai https://www.makaan.com/price-trends/property-rates-for-buy-in-mumbai
Pune https://www.makaan.com/price-trends/property-rates-for-buy-in-pune
Puri https://www.makaan.com/price-trends/property-rates-for-buy-in-puri
Bangalore https://www.makaan.com/price-trends/property-rates-for-buy-in-bangalore


In [7]:
locality_prices = scrape_cities(CITIES, CITY_URLS)

91 found for city : Chennai


91/91 for city Chennai: 100%|██████████| 91/91 [03:17<00:00,  2.17s/it]


102 found for city : Mumbai


102/102 for city Mumbai: 100%|██████████| 102/102 [03:41<00:00,  2.17s/it]


55 found for city : Pune


55/55 for city Pune: 100%|██████████| 55/55 [01:57<00:00,  2.13s/it]


1 found for city : Puri


1/1 for city Puri: 100%|██████████| 1/1 [00:01<00:00,  1.73s/it]


86 found for city : Bangalore


86/86 for city Bangalore: 100%|██████████| 86/86 [03:08<00:00,  2.19s/it]


In [8]:
locality_prices[0]

{'city_name': 'Chennai',
 'city_url': 'https://www.makaan.com/price-trends/property-rates-for-buy-in-chennai',
 'locality_name': 'Avadi',
 'price_range_per_sq_ft': ' 2,139 - 6,309 / sqft',
 'avg_price_per_sq_ft': ' 5,241.2 / sqft',
 'price_rise': '-5%',
 'type': 'apartment',
 'page': 1}

## Cleaning the data

- Convert ``price_range_per_sq_ft`` to 2 columns, ``price_range_per_sq_ft_low`` and ``price_range_per_sq_ft_high``
- Convert ``avg_price_per_sq_ft`` to ``float`` from ``string``
- Convert ``price_rise`` to ``float`` from ``string``

In [9]:
def clean_price_rise(s) : 
  s = s.strip()
  s = re.findall(r"[-+]?(?:\d*\.*\d+)", s)
  if len(s) == 0 : 
    return None
  else : 
    return float(s[0])

assert clean_price_rise("54%") == 54, "Wrong cleaning"
assert clean_price_rise("43.5%") == 43.5, "Wrong cleaning"
assert clean_price_rise("-0.9%") == -0.9, "Wrong cleaning"
assert clean_price_rise("-") == None, "Wrong cleaning"
print("All Passed")

All Passed


In [10]:
def clean_average(s) : 
  s = s.strip()
  s = re.sub(",", "", s)
  s = re.findall(r"(?:\d*\.*\d+)", s)
  if len(s) == 0 : 
    return None
  else : 
    return float(s[0])

assert clean_average("5,241.2 / sqft") == 5241.2, "Wrong cleaning"
assert clean_average("15,241,123.2 / sqft") == 15241123.2, "Wrong cleaning"
assert clean_average("5248 / sqft") == 5248, "Wrong cleaning"
assert clean_average("-") == None, "Wrong cleaning"
print("All Passed")

All Passed


In [11]:
def clean_range(s) : 
  s = s.strip()
  s = re.sub(",", "", s)
  s = re.findall(r"(?:\d*\.*\d+)", s)
  if len(s) == 0 : 
    return None, None
  elif len(s) == 1 : 
    return float(s[0]), float(s[0])
  else : 
    return float(s[0]), float(s[1])

assert clean_range("2,139 - 6,309 / sqft") == (2139, 6309), "Wrong cleaning"
assert clean_range("21.23 - 6,309 / sqft") == (21.23, 6309), "Wrong cleaning"
assert clean_range("58 / sqft") == (58, 58), "Wrong cleaning"
assert clean_range("-") == (None, None), "Wrong cleaning"
print("All Passed")

All Passed


In [12]:
def process_data(data) :
  '''
    Takes in a raw dataframe and preprocesses it
  '''
  dt = []
  for i in tqdm(data) :
    row = i.copy()
    low, high = clean_range(row["price_range_per_sq_ft"])
    row["price_range_per_sq_ft_low"] = low
    row["price_range_per_sq_ft_high"] = high
    del row["price_range_per_sq_ft"]
    row["avg_price_per_sq_ft"] = clean_average(row["avg_price_per_sq_ft"])
    row["price_rise"] = clean_price_rise(row["price_rise"])
    dt.append(row)
  return dt

### Cleaning cities data

In [13]:
city_prices[0]

{'city_name': 'Ahmedabad',
 'city_url': 'https://www.makaan.com/price-trends/property-rates-for-buy-in-ahmedabad',
 'price_range_per_sq_ft': ' 611 - 1,71,969 / sqft',
 'avg_price_per_sq_ft': ' 37,742.18 / sqft',
 'price_rise': '84.3%',
 'type': 'apartment'}

In [14]:
city_prices2 = process_data(city_prices)
city_prices2[0]

100%|██████████| 1808/1808 [00:00<00:00, 79964.80it/s]


{'city_name': 'Ahmedabad',
 'city_url': 'https://www.makaan.com/price-trends/property-rates-for-buy-in-ahmedabad',
 'avg_price_per_sq_ft': 37742.18,
 'price_rise': 84.3,
 'type': 'apartment',
 'price_range_per_sq_ft_low': 611.0,
 'price_range_per_sq_ft_high': 171969.0}

### Cleaning localities data

In [15]:
locality_prices[0]

{'city_name': 'Chennai',
 'city_url': 'https://www.makaan.com/price-trends/property-rates-for-buy-in-chennai',
 'locality_name': 'Avadi',
 'price_range_per_sq_ft': ' 2,139 - 6,309 / sqft',
 'avg_price_per_sq_ft': ' 5,241.2 / sqft',
 'price_rise': '-5%',
 'type': 'apartment',
 'page': 1}

In [16]:
locality_prices2 = process_data(locality_prices)
locality_prices2[0]

100%|██████████| 80128/80128 [00:00<00:00, 116947.92it/s]


{'city_name': 'Chennai',
 'city_url': 'https://www.makaan.com/price-trends/property-rates-for-buy-in-chennai',
 'locality_name': 'Avadi',
 'avg_price_per_sq_ft': 5241.2,
 'price_rise': -5.0,
 'type': 'apartment',
 'page': 1,
 'price_range_per_sq_ft_low': 2139.0,
 'price_range_per_sq_ft_high': 6309.0}

### Saving the cleaned data

In [17]:
save_to_csv(city_prices2, "makaan_city_prices.csv")

In [18]:
save_to_csv(locality_prices2, "makaan_locality_prices.csv")

## Converting the data to sqlite db

In [19]:
def save_data_to_db(db_file, data):
    """ 
      Create a db file and save the data in it
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        c = conn.cursor()
        # create table
        c.execute('''CREATE TABLE IF NOT EXISTS locations (
                        id integer PRIMARY KEY,
                        city_name text NOT NULL,
                        city_url text NOT NULL,
                        locality_name text NOT NULL,
                        price_range_per_sq_ft_low float,
                        price_range_per_sq_ft_high float,
                        avg_price_per_sq_ft float,
                        price_rise float,
                        type text NOT NULL,
                        page integer NOT NULL
                   );''')
        
        # insert data in the db file
        l = len(data)
        for id in tqdm(range(l)) :
          row = data[id]
          c.execute('''INSERT INTO locations(id,city_name,city_url,
                        locality_name,price_range_per_sq_ft_low,
                        price_range_per_sq_ft_high,avg_price_per_sq_ft,
                        price_rise,type,page) VALUES(?,?,?,?,?,?,?,?,?,?);
                  ''', (id, row['city_name'], row['city_url'], 
                        row['locality_name'], row['price_range_per_sq_ft_low'],
                        row['price_range_per_sq_ft_high'],
                        row['avg_price_per_sq_ft'], row['price_rise'],
                        row['type'], row['page']))
        conn.commit()
    except Error as e:
        print(e)
    finally:
        if conn:
            conn.close()

save_data_to_db(r"makaan_locality_prices.db", locality_prices2)

100%|██████████| 80128/80128 [00:00<00:00, 155642.00it/s]


## Create Aggregation of Data

In [20]:
data = []

city_pages = {city:set() for city in CITY_URLS.keys()}
city_localities = {city:set() for city in CITY_URLS.keys()}

for row in locality_prices2 : 
  city = row["city_name"]
  if city in CITY_URLS : 
    city_pages[city].add(row["page"])
    city_localities[city].add(row["locality_name"])

for city, url in CITY_URLS.items() : 
  data.append({
      "city_name" : city,
      "city_url" : url,
      "total_pages" : len(city_pages[city]),
      "total_localities": len(city_localities[city]) 
  })

In [21]:
save_to_csv(data, "makaan_localities.csv")

## Saving the data on drive

In [22]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [23]:
!cp *.csv /gdrive/MyDrive/TealTaskV2/
!cp makaan_locality_prices.db /gdrive/MyDrive/TealTaskV2/